# Нормализация вики

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('wiki_dataset/rus_wikitext_50k_caption_reference_description_train.csv', index_col=0)

In [4]:
df.head()

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,ru,https://ru.wikipedia.org/wiki/%D0%A3%D0%BC%D0%...,https://upload.wikimedia.org/wikipedia/commons...,Умершие в ноябре 2013 года,16 ноября,Умершие в ноябре 2013 года / 16 ноября,Владислав Захаревич,Русский: Ректор ЮФУ В.Г. ЗахаревичEnglish: Pro...,NaN,image/jpeg,1533,1677,False,True,True,"Это список известных людей, соответствующих ус...","Берберян, Арсен (75) — архиепископ Армянской а..."
5,ru,https://ru.wikipedia.org/wiki/%D0%91%D1%83%D1%...,https://upload.wikimedia.org/wikipedia/commons...,Буш (курган),NaN,Буш (курган),Зарисовка крупнейшего из Золотых ромбов курган...,English: design of the the Bush Barrow Lozenge...,NaN,image/jpeg,588,625,False,False,True,"Курган Буш, англ. Bush Barrow — памятник ранне...","Курган Буш, англ. Bush Barrow — памятник ранне..."
8,ru,https://ru.wikipedia.org/wiki/%D0%A2%D1%80%D0%...,https://upload.wikimedia.org/wikipedia/commons...,Трассовый моделизм,NaN,Трассовый моделизм,Трассовый моделизм. Прохождение моделями виража,日本語: YSR race Mr.ky,NaN,image/jpeg,540,960,False,False,False,Трассовый моделизм — вид автомодельного спорта...,Трассовый моделизм (также трассовые гонки или ...
15,ru,https://ru.wikipedia.org/wiki/%D0%9B%D0%B8%D0%...,https://upload.wikimedia.org/wikipedia/commons...,Лилейная трещалка,NaN,Лилейная трещалка,Спаривание,"Liloceris lilii (Scopoli, 1763), copula Englis...",NaN,image/jpeg,2336,3504,False,False,True,Треща́лка лиле́йная — насекомое семейства жуко...,Треща́лка лиле́йная (лат. Lilioceris lilii) — ...
16,ru,https://ru.wikipedia.org/wiki/%D0%91%D1%83%D1%...,https://upload.wikimedia.org/wikipedia/commons...,"Бурылин, Дмитрий Геннадьевич",Коллекционная деятельность,"Бурылин, Дмитрий Геннадьевич / Коллекционная д...",Музей Дмитрия Бурылина.,English: Museum of Industry and Art (Burylin M...,NaN,image/jpeg,3759,5012,False,True,False,Дми́трий Генна́дьевич Буры́лин — иваново-возне...,Всю свою жизнь Дмитрий Бурылин собирал коллекц...


In [5]:
df.shape

(41851, 17)

In [6]:
df = df.drop(columns=['page_changed_recently', 'page_changed_recently', 'page_url', 'image_url', 'mime_type'])

In [7]:
columns = ['page_title', 'section_title', 'hierarchical_section_title', 'caption_reference_description',
           'caption_attribution_description', 'context_page_description', 'context_section_description']

In [8]:
from tqdm.auto import tqdm
import re

In [9]:
df.loc[df.caption_reference_description.isna(), 'caption_alt_text_description'] = df.caption_alt_text_description

In [10]:
df.loc[df.caption_reference_description.isna(), 'page_title'] = df.page_title

In [11]:
df = df.loc[df.caption_reference_description.notna()]

In [11]:
df.index[:]

Int64Index([    0,     5,     8,    15,    16,    22,    23,    30,    32,
               33,
            ...
            49975, 49976, 49980, 49982, 49988, 49989, 49993, 49995, 49997,
            49999],
           dtype='int64', length=18598)

In [13]:
to_drop = []
total_count = 0
for i in tqdm(df.index[:]):
    text = df.caption_reference_description.loc[i]
    count = len(re.findall(r'\b\d+\b', text))
    if count > 0:
        total_count += 1
    if count > (1 / 3) * len(text):
        to_drop.append(i)
df = df.drop(to_drop)
print(total_count)
print(len(to_drop))

100%|██████████| 18598/18598 [00:00<00:00, 43550.41it/s]

6665
0


In [ ]:
lines_to_drop = []
for i in tqdm(df.index):
    for col in columns:
        df[col][i] = str(df[col][i])
        df[col][i] = re.sub("\S*\d\S*", "", df[col][i]).strip()
        df[col][i] = re.sub(r'(?i)[a-z]', '', df[col][i])
        df[col][i] = df[col][i].replace("г.", "").replace("№", "").replace("год", "")
        df[col][i] = " ".join(df[col][i].split())
        if len(df[col][i]) > 0 and df[col][i][-1] == ' ':
            df[col][i] = df[col][i][:-1]
        if len(df[col][i]) > 0 and df[col][i][-1] == ',':
            df[col][i] = df[col][i][:-1]
        if col == "caption_reference_description" and len(df[col][i]) == 0:
            lines_to_drop.append(i)
print(len(lines_to_drop))

In [13]:
df = df.drop(lines_to_drop)

In [14]:
len(df.loc[df.caption_reference_description == df.caption_alt_text_description]) / len(df) * 100

0.24273630011033467

In [15]:
df = df.drop(columns=['caption_alt_text_description'])

In [16]:
df.shape

(40785, 12)

In [17]:
df.to_csv('wiki_dataset/rus_wikitext_50k_caption_reference_description_train_norm.csv')

# Тестим (неактуально)

In [11]:
import torch

In [ ]:
!pip install torchmetrics == 0.11.3

In [ ]:
!pip install transformers == 4.26.1

In [ ]:
from torchmetrics.multimodal import CLIPScore

metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

In [27]:
import torchvision.transforms as transforms
from PIL import Image


def read_image(path):
    img = Image.open(path).convert('RGB')

    resize = transforms.Resize([224, 224])
    img = resize(img)
    to_tensor = transforms.ToTensor()

    tensor = to_tensor(img)
    return tensor

In [16]:
images = torch.load('wiki_img_tensor.pt')
wikiScore = metric(images[0], df.caption_reference_description.loc[0])

In [17]:
wikiScore.detach()

tensor(19.9878)

In [ ]:
scores_wiki = []
j = 0
for i in tqdm(df.index[:]):
    text = df.caption_reference_description.loc[i]
    try:
        wikiScore = metric(images[j], text)
        scores_wiki.append(wikiScore.detach())
    except KeyboardInterrupt:
        print("stop")
        break
    except:
        pass
    j += 1
mean_wiki = sum(scores_wiki) / len(scores_wiki)

In [20]:
mean_wiki  # оценка wiki

tensor(21.9187)

In [ ]:
import torchvision.datasets as dset

coco_train = dset.CocoDetection(root="coco_dataset/train2017",
                                annFile="coco_dataset/annotations/captions_train2017.json")

In [27]:
scores_coco = []
for i in tqdm(range(9278)):
    image, captions = coco_train[i]
    name = str(captions[0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2017/{name}.jpg"
    text = captions[0]['caption']
    cocoScore = metric(read_image(name), text)
    scores_coco.append(cocoScore.detach())
mean_coco = sum(scores_coco) / len(scores_coco)

100%|██████████| 9278/9278 [33:58<00:00,  4.55it/s]


In [28]:
mean_coco  # оценка coco

tensor(30.4789)

In [ ]:
!pip install googletrans == 3.1.0a0

In [ ]:
from googletrans import Translator

translator = Translator()
translated_text = []
for i in tqdm(range(5000)):
    image, captions = coco_train[i]
    text = captions[0]['caption']
    translated_text.append(translator.translate(text, dest="ru", src="en").text)

In [68]:
scores_coco = []
for i in tqdm(range(1000)):
    image, captions = coco_train[i]
    name = str(captions[0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2017/{name}.jpg"
    text = translated_text[i]
    try:
        cocoScore = metric(read_image(name), text)
        if cocoScore.detach() > 15:
            scores_coco.append(cocoScore.detach())
    except:
        pass
mean_coco = sum(scores_coco) / len(scores_coco)

100%|██████████| 1000/1000 [04:04<00:00,  4.09it/s]


In [69]:
len(scores_coco) / 1000  # процент норм переведенных описаний

0.913

In [ ]:
import torchvision.datasets as dset

coco_train_russian = dset.CocoDetection(root="coco_dataset/train2014",
                                        annFile="coco_dataset/annotations/captions_train2014.json")

In [24]:
import json

with open('coco_dataset/captions_ru_train.json', 'r') as f:
    coco_captions_train_russian = json.load(f)

In [38]:
from tqdm.auto import tqdm

scores_coco_russian = []
for i in tqdm(range(1000)):
    image = coco_train_russian[i][0]
    captions = coco_captions_train_russian[i]
    name = str(coco_train_russian[i][1][0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2014/COCO_train2014_{name}.jpg"
    image = read_image(path=name)

    text = captions[0]['caption']

    try:
        cocoScore = metric(read_image(name), text)
        scores_coco_russian.append(cocoScore.detach())
    except:
        pass
mean_coco_russian = sum(scores_coco_russian) / len(scores_coco_russian)

100%|██████████| 1000/1000 [04:13<00:00,  3.95it/s]


In [39]:
mean_coco_russian  # оценка rucoco

tensor(19.0822)

# Перевод coco

In [67]:
mean_coco  # оценка переведенного coco

tensor(18.9519)

In [19]:
n = 20000
coco_translation = []

In [25]:
len(coco_translation)

11218

In [ ]:
from googletrans import Translator
import json

translator = Translator()
for i in tqdm(range(len(coco_translation), n)):
    image, captions = coco_train[i]
    name = str(captions[0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2017/{name}.jpg"
    translated_captions = []
    for cur_cap in captions:
        text = translator.translate(cur_cap['caption'], dest="ru", src="en").text
        try:
            cocoScore = metric(read_image(name), text)
            if cocoScore.detach() > 15:
                translated_captions.append(text)
        except:
            pass
    coco_translation.append(translated_captions)
    if i % 1000 == 0:
        with open('coco_dataset/coco_captions_russian.json', 'w', encoding='utf-8') as f:
            json.dump(coco_translation, f, ensure_ascii=False, indent=4)

In [24]:
with open('coco_dataset/coco_captions_russian.json', 'w', encoding='utf-8') as f:
    json.dump(coco_translation, f, ensure_ascii=False, indent=4)

# Тестим 2

In [ ]:
from multilingual_clip import pt_multilingual_clip
import transformers
import torch
import open_clip
import requests
from PIL import Image

model_name = 'M-CLIP/XLM-Roberta-Large-Vit-B-16Plus'

model_caption = pt_multilingual_clip.MultilingualCLIP.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [2]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"
model_img, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model_img.to(device)
None

In [3]:
def compare_embeddings(logit_scale, img_embs, txt_embs):
    # normalized features
    image_features = img_embs / img_embs.norm(dim=-1, keepdim=True)
    text_features = txt_embs / txt_embs.norm(dim=-1, keepdim=True)

    # cosine similarity as logits
    logits_per_image = logit_scale * image_features @ text_features.t()
    logits_per_text = logit_scale * text_features @ image_features.t()

    # shape = [global_batch_size, global_batch_size]
    return logits_per_image, logits_per_text

In [4]:
logit_scale = model_img.logit_scale.exp().float()

# ruCOCO

In [ ]:
import torchvision.datasets as dset

coco_train_russian = dset.CocoDetection(root="coco_dataset/train2014",
                                        annFile="coco_dataset/annotations/captions_train2014.json")

In [5]:
import json

with open('coco_dataset/captions_ru_train.json', 'r') as f:
    coco_captions_train_russian = json.load(f)

In [ ]:
len(coco_train_russian)

82783

In [7]:
from tqdm.auto import tqdm

russian_captions, demo_images = [], []
for i in tqdm(range(len(coco_train_russian))):
    image = coco_train_russian[i][0]
    captions = coco_captions_train_russian[i]
    text = captions[0]['caption']
    russian_captions.append(text)

    name = str(coco_train_russian[i][1][0]['image_id'])
    name = "0" * (12 - len(name)) + name
    name = f"coco_dataset/train2014/COCO_train2014_{name}.jpg"
    demo_images.append(name)

100%|██████████| 82783/82783 [10:37<00:00, 129.88it/s]


In [8]:
import time

model_caption.eval()
embeddings = torch.Tensor([])
for i in tqdm(range(828)):
    with torch.no_grad():
        embeddings = torch.cat(
            [embeddings, model_caption.forward(russian_captions[i * 100: (i + 1) * 100], tokenizer).detach().cpu()])

100%|██████████| 828/828 [46:58<00:00,  3.40s/it]  


In [9]:
embeddings.shape

torch.Size([82783, 640])

In [16]:
from multilingual_clip import pt_multilingual_clip
import transformers
import torch
import numpy as np

score = []
for i, name in tqdm(enumerate(demo_images), total=len(demo_images)):
    with torch.no_grad():
        with Image.open(name) as curimg:
            images = preprocess(curimg).unsqueeze(0)
        image_features = model_img.encode_image(images.to(device)).detach().float()
        img_logits, text_logits = compare_embeddings(logit_scale, image_features, embeddings[i].to(device))
        score.append(img_logits)
        #print(img_logits)

100%|██████████| 82783/82783 [23:46<00:00, 58.03it/s]


In [17]:
torch.cat(score).mean()

tensor(31.1980, device='cuda:0')

# Wiki

In [5]:
import pandas as pd

df = pd.read_csv('wiki_dataset/rus_wikitext_50k_caption_reference_description_train_norm.csv', index_col=0)

In [6]:
len(df)

40785

In [7]:
wiki_captions = df['caption_reference_description']
wiki_captions[:15]

0                                   Владислав Захаревич
5     Зарисовка крупнейшего из Золотых ромбов курган...
8       Трассовый моделизм. Прохождение моделями виража
15                                           Спаривание
16                              Музей Дмитрия Бурылина.
22                        Павильон «Космос». Фотография
23    Премудрый царь Берендей. Эскиз Виктора Васнецо...
30                           Герб Сталь-фон-Гольштейнов
32                            Памятник Иванову в Томске
33    Поскольку изменение климата приводит к таянию ...
40                            Филипп Йорк, граф Хардвик
49           Подарок зубной фее от восьмилетней девочки
57        Факультет медицины Сельджукского университета
59                Пинхас Лавон, министр обороны Израиля
61            Странноприимный дом Таранова-Белозёрова (
Name: caption_reference_description, dtype: object

In [ ]:
model_caption.eval()
wiki_embeddings = torch.Tensor([])
for i, num in tqdm(enumerate(wiki_captions.keys()), total=len(wiki_captions)):
    try:
        with torch.no_grad():
            wiki_embeddings = torch.cat(
                [wiki_embeddings, model_caption.forward(str(wiki_captions[num]), tokenizer).detach().cpu()])
    except:
        wiki_embeddings = torch.cat([wiki_embeddings, torch.zeros(640)])
        pass

In [ ]:
wiki_score = []
for i, num in tqdm(enumerate(df.index), total=len(df)):
    try:
        with torch.no_grad():
            with Image.open(f"wiki_dataset/images/image_{num}.jpg") as curimg:
                imgs = preprocess(curimg).unsqueeze(0)
            image_features = model_img.encode_image(imgs.to(device)).detach().float()
            img_logits, text_logits = compare_embeddings(logit_scale, image_features, wiki_embeddings[i].to(device))
            wiki_score.append(img_logits)
    except:
        pass

In [134]:
torch.cat(wiki_score).mean()

tensor(24.7229, device='cuda:0')

In [139]:
more_than_10 = 0
more_than_15 = 0
more_than_20 = 0
more_than_25 = 0

for i in range(len(wiki_score)):
    if wiki_score[i] > 25:
        more_than_25 += 1
    elif wiki_score[i] > 20:
        more_than_20 += 1
    elif wiki_score[i] > 15:
        more_than_15 += 1
    elif wiki_score[i] > 10:
        more_than_10 += 1
print("(10; 15]:", more_than_10)
print("(15; 20]:", more_than_15)
print("(20; 25]:", more_than_20)
print("(25; 100]:", more_than_25)

(10; 15]: 1414
(15; 20]: 2153
(20; 25]: 3312
(25; 100]: 9214


In [136]:
wiki_score[:15]

[tensor([17.5509], device='cuda:0'),
 tensor([16.8161], device='cuda:0'),
 tensor([22.0616], device='cuda:0'),
 tensor([7.3211], device='cuda:0'),
 tensor([39.4286], device='cuda:0'),
 tensor([18.0624], device='cuda:0'),
 tensor([34.1346], device='cuda:0'),
 tensor([35.0481], device='cuda:0'),
 tensor([25.1546], device='cuda:0'),
 tensor([36.3066], device='cuda:0'),
 tensor([13.8209], device='cuda:0'),
 tensor([15.6998], device='cuda:0'),
 tensor([16.4227], device='cuda:0'),
 tensor([22.5956], device='cuda:0'),
 tensor([14.4938], device='cuda:0')]

In [8]:
from tqdm.auto import tqdm

img_features = {}
for i, num in tqdm(enumerate(wiki_captions.keys()), total=len(wiki_captions)):
    try:
        with Image.open(f"wiki_dataset/images/image_{num}.jpg") as curimg:
            imgs = preprocess(curimg).unsqueeze(0)
        img_features[num] = model_img.encode_image(imgs.to(device)).detach().float()
    except:
        wiki_captions.pop(num)
        pass

torch.save(img_features, 'wiki_img_features.pt')

In [92]:
img_features = torch.load('wiki_img_features.pt')

In [86]:
sentences = []


def delete_quotes(k, i):
    k = k[:i] + k[k[i:].find('»') + i + 1:]
    k = " ".join(k.split())
    return k


def balance(s):
    ans = 0
    for i in s:
        if i == "«":
            ans += 1
        elif i == "»":
            ans -= 1
    return ans


def fix_quotes(s):
    st = []
    to_del = []
    for i in range(len(s)):
        if s[i] == "«":
            st.append([i, 0])
        else:
            if len(st) > 0 and st[-1][1] == 0:
                st.pop()
            else:
                to_del.append(i)
    for i in st:
        to_del.append(i[0])
    fixed_s = ""
    for i in range(len(s)):
        if i not in to_del:
            fixed_s += s[i]
    return fixed_s


def fight_with_quotes(s):
    sentences.append(s)
    if balance(s) != 0:
        fix_quotes(s)
    for i in range(len(s)):
        if s[i] == "«":
            t = delete_quotes(s, i)
            if t not in sentences:
                fight_with_quotes(t)
    return

In [99]:
import regex as re
from PIL import Image
from tqdm.auto import tqdm
import json

model_caption.eval()
wiki_score = []
good_wiki_captions = []

split_regex = re.compile(r'[.|!|?|…]')
for i, num in tqdm(enumerate(wiki_captions.keys()), total=len(wiki_captions)):
    try:
        image_features = img_features[num]
    except KeyError:
        continue
    sentences = []
    splitted_text = filter(lambda t: t, [t.strip() for t in split_regex.split(str(wiki_captions[num]))])
    for sent in splitted_text:
        fight_with_quotes(sent)
    cur_emb = []
    for j, s in enumerate(sentences):
        with torch.no_grad():
            emb = model_caption.forward(s, tokenizer).detach().to(device)
        img_logits, text_logits = compare_embeddings(logit_scale, image_features, emb)
        if img_logits >= 15:
            cur_emb.append([img_logits, j])
    if len(cur_emb) > 0:
        best_emb = max(cur_emb)
        wiki_score.append(best_emb[0])
        good_wiki_captions.append([sentences[best_emb[1]], num])
    if i % 100 == 0:
        torch.save(wiki_score, 'wiki_score.pt')
        with open("good_wiki_captions.json", "w", encoding="utf-8") as file:
            json.dump(good_wiki_captions, file)

100%|██████████| 40785/40785 [1:28:53<00:00,  7.65it/s]  


In [100]:
torch.cat(wiki_score).mean()

tensor(27.0535, device='cuda:1', grad_fn=<MeanBackward0>)

In [4]:
import json

with open("good_wiki_captions.json", "r", encoding="utf-8") as file:
    good_wiki_captions = json.load(file)

In [8]:
len(good_wiki_captions)

35111

In [9]:
good_wiki_captions[:10]

[['Зарисовка крупнейшего из Золотых ромбов кургана Буш', 5],
 ['Трассовый моделизм', 8],
 ['Музей Дмитрия Бурылина', 16],
 ['Павильон «Космос»', 22],
 ['Эскиз Виктора Васнецова к опере', 23],
 ['Герб Сталь-фон-Гольштейнов', 30],
 ['Памятник Иванову в Томске', 32],
 ['Поскольку изменение климата приводит к таянию морского льда, Геологическая служба США прогнозирует, что две трети полярных медведей исчезнут к',
  33],
 ['Филипп Йорк, граф Хардвик', 40],
 ['Подарок зубной фее от восьмилетней девочки', 49]]

In [109]:
for i, j in zip(good_wiki_captions[:100], wiki_score[:100]):
    print(i, j[0][0])

['Зарисовка крупнейшего из Золотых ромбов кургана Буш', 5] tensor(16.5067, device='cuda:1', grad_fn=<SelectBackward0>)
['Трассовый моделизм', 8] tensor(23.3505, device='cuda:1', grad_fn=<SelectBackward0>)
['Музей Дмитрия Бурылина', 16] tensor(36.0324, device='cuda:1', grad_fn=<SelectBackward0>)
['Павильон «Космос»', 22] tensor(23.6495, device='cuda:1', grad_fn=<SelectBackward0>)
['Эскиз Виктора Васнецова к опере', 23] tensor(29.3526, device='cuda:1', grad_fn=<SelectBackward0>)
['Герб Сталь-фон-Гольштейнов', 30] tensor(36.4239, device='cuda:1', grad_fn=<SelectBackward0>)
['Памятник Иванову в Томске', 32] tensor(26.2568, device='cuda:1', grad_fn=<SelectBackward0>)
['Поскольку изменение климата приводит к таянию морского льда, Геологическая служба США прогнозирует, что две трети полярных медведей исчезнут к', 33] tensor(36.0533, device='cuda:1', grad_fn=<SelectBackward0>)
['Филипп Йорк, граф Хардвик', 40] tensor(22.2617, device='cuda:1', grad_fn=<SelectBackward0>)
['Подарок зубной фее от 